# T-SQL in a SQL Notebook

Now that we can add some PowerShell to our Notebooks, I can show you how to add some T-SQL.

First lets create a container to use for our notebook

I am going to use the dbatools docker images for this. You can read [more about that here](https://dbatools.io/docker/)

I am going to use the docker-compose file from [my DockerStuff repo](https://github.com/SQLDBAWithABeard/DockerStuff/tree/master/dbatools-2-instances-AG) which I have saved locally. This uses a local directory for backup storage.

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 if (-not (%24IsLinux -or %24IsMacOS) ) \u000A%7b \u000A    if (-not(Test-Path C:%5c%5c%5c%5cMSSQL%5c%5c%5c%5cBACKUP)) \u000A    %7b \u000A        Write-Output \u0027I\u0027\u0027m going to create C:%5c%5c%5c%5cMSSQL%5c%5c%5c%5cBACKUPS so that the docker-compose will work\u0027\u000A        New-Item C:%5c%5cMSSQL1%5c%5cBACKUP1 -ItemType Directory \u000A    %7d    else %7b \u000A        Write-Output \u0027C:%5c%5c%5c%5cMSSQL%5c%5c%5c%5cBACKUPS already exists\u0027 %7d \u000A%7d elseif(-not %24IsMacOS)%7b\u000A    if (-not(Test-Path %2ftmp%2fdocker%2fvolumes%2fdbatools)) \u000A    %7b \u000A        Write-Output \u0027I\u0027\u0027m going to create %2ftmp%2fdocker%2fvolumes%2fdbatools so that the docker-compose will work\u0027\u000A        New-Item %2ftmp%2fdocker%2fvolumes%2fdbatools -ItemType Directory \u000A    %7d    else %7b \u000A        Write-Output \u0027%2ftmp%2fdocker%2fvolumes%2fdbatools already exists\u0027 %7d \u000A%7d else %7b Write-Warning \u0027Sorry This code won\u0027\u0027t run on Mac - You will have to create the folder manually and edit the docker compose file\u0027 %7d \u000A \u000D %22%7D">This link will create the volumes for the containers to work</a>

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 docker-compose -f C:%5C%5Cdbachecks%5C%5Cdbatools-docker-compose.yml up -d \u000D %22%7D">This will run the docker compose and create the containers</a>

![image](https://user-images.githubusercontent.com/6729780/59587466-39003d80-90dd-11e9-8ca7-2c1550cb96cb.png)






# T-SQL

You can add T-SQL Code by clicking on the three elipses in a block

![image](https://user-images.githubusercontent.com/6729780/59587952-600b3f00-90de-11e9-9801-d4c7664aaa84.png)

or by using the button at the top of the Notebook

![image](https://user-images.githubusercontent.com/6729780/59588101-c001e580-90de-11e9-8f86-f6b522c69322.png)

Either of these creates a Code Block. You can enter your T-SQL code in the block

![image](https://user-images.githubusercontent.com/6729780/59588323-3dc5f100-90df-11e9-85cf-9b9d5ae0c877.png)

You can set a connection at the top of the notebook. It defaults to the first instance that you have connected to in Azure Data Studio

![image](https://user-images.githubusercontent.com/6729780/59588669-0c99f080-90e0-11e9-95a0-95710f67b3b9.png)

You can click the play button or F5 to run the code

![image](https://user-images.githubusercontent.com/6729780/59588515-aca34a00-90df-11e9-9c76-6b9075ceca45.png)

To change database you will `USE [DatabaseName]`


In [1]:
SELECT @@VERSION AS SQLVersion

In [1]:
USE Northwind
GO

SELECT c.CategoryName,
  (select sum(val)
   from (SELECT TOP 5 od2.UnitPrice*od2.Quantity as val
         FROM [Order Details] od2, Products p2
         WHERE od2.ProductID = p2.ProductID
         AND c.CategoryID = p2.CategoryID
         ORDER BY 1 DESC
        ) t
  ) AS "5 top orders in 1997"
FROM [Order Details] od, Products p, Categories c, Orders o 
WHERE od.ProductID = p. ProductID
AND p.CategoryID = c.CategoryID
AND od.OrderID = o.OrderID
AND YEAR(o.OrderDate) = 1997
GROUP BY c.CategoryName, c.CategoryId

Commands completed successfully.

(8 rows affected)

Total execution time: 00:00:00.212

CategoryName,5 top orders in 1997
Beverages,63240.0000
Condiments,12989.5000
Confections,19379.0000
Dairy Products,21830.0000
Grains/Cereals,13822.5000
Meat/Poultry,35899.1000
Produce,17638.0000
Seafood,13082.5000


# Results

The results are saved with the notebook. This means that you can send the notebook to your clients, give it to your off-site engineers or first line support and you will then know *exactly* which queries they ran and the results of that query. So you can easily create runbooks for certain scenarios.

You could also use notebooks for upgrades or migrations and have a recorded timeline of what happened.

That would also be good for incident resolution - Let your mind go wild :-)

# Availability Groups

One thing that you could do is to use code blocks for users to follow to perform repeated tasks like maybe creating Availability Groups.

I was going to create a demo showing exaclty that but actually its so much easierr to do it with dbatools I thought I would do that instead!

This is the code

    $cred = Import-Clixml -Path dockercompose:\dbatools-2-instances-AG\sqladmincred.xml
    $sqlinstance1 = 'localhost,15591'
    $sqlinstance2 = 'localhost,15592'
    $AGName = 'dbatools-ag'

    $params = @{
        Primary = $sqlinstance1
        PrimarySqlCredential = $cred
        Secondary = $sqlinstance2
        SecondarySqlCredential = $cred
        Name = $AGName
        Database = 'pubs'
        ClusterType = 'None'
        SeedingMode = 'Automatic'
        FailoverMode = 'Manual'
        Confirm = $false
    }
    
    # execute the command
    New-DbaAvailabilityGroup @params 

<a href="command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22 \u000ASystem.Management.Automation.PSCredential %3d Import-Clixml -Path %2fhome%2frob%2fDocuments%2fgit%2fDockerStuff%2fdbatools-2-instances-AG%2flinux%2fsqladmin.cred\u000Alocalhost%2c15591 %3d \u0027localhost%2c15591\u0027\u000Alocalhost%2c15592 %3d \u0027localhost%2c15592\u0027\u000Adbatools-ag %3d \u0027dbatools-ag\u0027\u000ASystem.Collections.Hashtable %3d %40%7b\u000A    Primary %3d localhost%2c15591\u000A    PrimarySqlCredential %3d System.Management.Automation.PSCredential\u000A    Name %3d dbatools-ag\u000A    Database %3d \u0027pubs\u0027\u000A    ClusterType %3d \u0027External\u0027\u000A    SeedingMode %3d \u0027Automatic\u0027\u000A    FailoverMode %3d \u0027External\u0027\u000A    Confirm %3d False\u000A%7d\u000ASecondary %3d localhost%2c15592\u000ASecondarySqlCredential %3d System.Management.Automation.PSCredential\u000A%23 execute the command\u000ANew-DbaAvailabilityGroup %40params \u000AGet-DbaAvailabilityGroup -SqlInstance localhost%2c15591 -SqlCredential System.Management.Automation.PSCredential %7c Add-DbaAgReplica -SqlInstance localhost%2c15592 -SqlCredential System.Management.Automation.PSCredential -FailoverMode External \u000D %22%7D">Lets create an AG - This won't work very well on Linux in docker</a>

I don't know why the AG DB Status doesnt show in the report but I can show it with T-SQL

(See how we mix PowerShell, Markdown and T-SQL to create a run book ?)

In [3]:
SELECT 
	ar.replica_server_name, 
	adc.database_name, 
	ag.name AS ag_name, 
	drs.is_local, 
	drs.is_primary_replica, 
	drs.synchronization_state_desc, 
	drs.is_commit_participant, 
	drs.synchronization_health_desc, 
	drs.recovery_lsn, 
	drs.truncation_lsn, 
	drs.last_sent_lsn, 
	drs.last_sent_time, 
	drs.last_received_lsn, 
	drs.last_received_time, 
	drs.last_hardened_lsn, 
	drs.last_hardened_time, 
	drs.last_redone_lsn, 
	drs.last_redone_time, 
	drs.log_send_queue_size, 
	drs.log_send_rate, 
	drs.redo_queue_size, 
	drs.redo_rate, 
	drs.filestream_send_rate, 
	drs.end_of_log_lsn, 
	drs.last_commit_lsn, 
	drs.last_commit_time
FROM sys.dm_hadr_database_replica_states AS drs
INNER JOIN sys.availability_databases_cluster AS adc 
	ON drs.group_id = adc.group_id AND 
	drs.group_database_id = adc.group_database_id
INNER JOIN sys.availability_groups AS ag
	ON ag.group_id = drs.group_id
INNER JOIN sys.availability_replicas AS ar 
	ON drs.group_id = ar.group_id AND 
	drs.replica_id = ar.replica_id
ORDER BY 
	ag.name, 
	ar.replica_server_name, 
	adc.database_name;

(2 rows affected)

Total execution time: 00:00:00.074

replica_server_name,database_name,ag_name,is_local,is_primary_replica,synchronization_state_desc,is_commit_participant,synchronization_health_desc,recovery_lsn,truncation_lsn,last_sent_lsn,last_sent_time,last_received_lsn,last_received_time,last_hardened_lsn,last_hardened_time,last_redone_lsn,last_redone_time,log_send_queue_size,log_send_rate,redo_queue_size,redo_rate,filestream_send_rate,end_of_log_lsn,last_commit_lsn,last_commit_time
36f57610fa49,pubs,dbatools-ag,0,0,NOT SYNCHRONIZING,0,NOT_HEALTHY,1,34000000035200038,1,2019-08-31 08:08:20.787,1,2019-08-31 08:08:20.787,0,2019-08-31 08:08:20.787,0,2019-08-31 08:08:20.787,NULL,0,0,0,0,0,0,NULL
82595dbaac28,pubs,dbatools-ag,1,1,SYNCHRONIZED,1,HEALTHY,4294967295429496729500001,34000000035200038,NULL,NULL,NULL,NULL,37000000024800001,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,37000000024000001,37000000020000025,2019-08-31 08:04:24.667


In [5]:
SELECT @@version AS 'Hugo thinks differently'